In [1]:
import pandas as pd

C:\Users\ksbso\AppData\Local\Temp\ipykernel_19712\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
domain_mx = pd.read_csv("data/gov_mx_results.csv").drop_duplicates()
mx_ip = pd.read_csv("data/gov_mailserver_a_results.csv").drop_duplicates()
ip_geodata = pd.read_csv("data/gov_mailserver_geo.csv").drop_duplicates()

In [3]:
data_mx_ip = pd.merge(domain_mx, mx_ip, how="left", left_on="exchange", right_on="url")
data_mx_ip = data_mx_ip.drop(columns=["url_y"]).rename(columns={"url_x": "domain"})
data_ip_geo = pd.merge(mx_ip, ip_geodata, how="left", left_on="ipv4", right_on="ip")
data_ip_geo = data_ip_geo.drop(columns=["ip"])

In [4]:
gov_domains = domain_mx.url.unique()
mailserver_domains = domain_mx.exchange.unique()
mailserver_ips = mx_ip.ipv4.unique()
mailserver_coords = ip_geodata[["country_name", "latitude", "longitude"]].drop_duplicates()

print("Unique .gov domains: ", len(gov_domains))
print("Unique Mailserver domains: ", len(mailserver_domains))
print("Unique Mailserver IPs: ", len(mailserver_ips))
print("Unique Mailserver locations: ", mailserver_coords.shape[0])
print("Locations in US: ", mailserver_coords[mailserver_coords["country_name"] == "United States"].shape[0])

Unique .gov domains:  7391
Unique Mailserver domains:  7471
Unique Mailserver IPs:  2282
Unique Mailserver locations:  524
Locations in US:  504


In [5]:
try:  
    mailserver_coords= mailserver_coords.reset_index()
except:
    print("already reset!")

locations = []
location_to_ip = dict()
for i in range(0, mailserver_coords.shape[0]):
    loc = mailserver_coords.loc[i]
    ips = ip_geodata[ip_geodata["latitude"] == loc["latitude"]]
    ips = ips[ips["longitude"] == loc["longitude"]]
    coord = loc["latitude"], loc["longitude"]
    locations.append(coord)
    location_to_ip[coord] = ips

In [6]:
loc_ip_data_summ = []
for location in locations:
    num = len(location_to_ip[location])
    loc_ip_data_summ.append([location, num])

loc_ip_data_summ_df = pd.DataFrame(loc_ip_data_summ, columns=['Location', '# IPs'])

In [35]:
valid_locations = loc_ip_data_summ_df.sort_values(by=['# IPs'], ascending=False).drop(4).drop(152).reset_index().drop(columns=["index"])
valid_locations.head(22)

,Location,# IPs
0,"(34.0544, -118.244)",169
1,"(39.0469, -77.4903)",79
2,"(45.8234, -119.7257)",41
3,"(39.9625, -83.0061)",38
4,"(45.1701, -93.1913)",38
5,"(43.6319, -79.3716)",38
6,"(37.1835, -121.7714)",23
7,"(49.2732, -123.0124)",16
8,"(38.7095, -78.1539)",13
9,"(41.6021, -93.6124)",12


In [29]:
print("Total IPs: ", valid_locations.shape[0])
print("Number of locations with greater than 5 IPs: ", valid_locations[valid_locations["# IPs"] > 5].shape[0])
print("Number of locations with 5 IPs: ", valid_locations[valid_locations["# IPs"] == 5].shape[0])
print("Number of locations with 4 IPs: ", valid_locations[valid_locations["# IPs"] == 4].shape[0])
print("Number of locations with 3 IPs: ", valid_locations[valid_locations["# IPs"] == 3].shape[0])
print("Number of locations with 2 IPs: ", valid_locations[valid_locations["# IPs"] == 2].shape[0])
print("Number of locations with 1 IP: ", valid_locations[valid_locations["# IPs"] == 1].shape[0])

Total IPs:  522
Number of locations with greater than 5 IPs:  22
Number of locations with 5 IPs:  4
Number of locations with 4 IPs:  6
Number of locations with 3 IPs:  31
Number of locations with 2 IPs:  100
Number of locations with 1 IP:  359
